### Essential Libraries

In [1]:

import torch
import csv
import os
import time
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
from dataloader import ATeX  # Ensure dataloader.py is in the same directory or in PYTHONPATH
from sklearn.metrics import classification_report
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import StepLR


### Dataset Preparation with Augmentation

In [2]:

# Define transformations with augmentation
mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean_std[0], mean_std[1]),
])

# Define datasets
train_dataset = ATeX(split="train", transform=data_transforms)
test_dataset = ATeX(split="test", transform=data_transforms)
val_dataset = ATeX(split="val", transform=data_transforms)

# Define DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


### Updated Custom CNN Model

In [3]:
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2)

        # Second convolutional layer
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(2)

        # Third convolutional layer
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(2)

        # Fourth convolutional layer
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool4 = nn.MaxPool2d(2)

        # Fully connected layers
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512 * 2 * 2, 1024)  # Adjust size based on input size
        self.bn5 = nn.BatchNorm1d(1024)
        self.dropout1 = nn.Dropout(0.1)

        self.fc2 = nn.Linear(1024, 512)
        self.bn6 = nn.BatchNorm1d(512)
        self.dropout2 = nn.Dropout(0.1)

        self.fc3 = nn.Linear(512, 15)  # Output layer (adjust for your class count)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool4(x)
        x = self.flatten(x)
        x = F.relu(self.bn5(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn6(self.fc2(x)))
        x = self.dropout2(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)


# Device configuration and model initialization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomCNN().to(device)


### Optimizer, Loss, and Scheduler

In [5]:
# Define optimizer, loss function, and learning rate scheduler
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)  # Changed to SGD with momentum
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)


### Training, Validation and Their Evaluation

In [6]:
# Initialize lists to store metrics
metrics = []

# Address to save the CSV file
save_dir = r"D:/USC_Course/CSCE 790 Section 007 Neural Networks and Their Applications/atex-main/loss_training time/"
save_path = os.path.join(save_dir, "training_metrics_custom_model.csv")

# Ensure the directory exists
os.makedirs(save_dir, exist_ok=True)

# Record the start time of the training process
training_start_time = time.time()

# Training loop with learning rate scheduler
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels, _ in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    all_val_labels, all_val_preds = [], []
    with torch.no_grad():
        for images, labels, _ in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            all_val_labels.extend(labels.cpu().numpy())
            all_val_preds.extend(preds.cpu().numpy())
    val_loss /= len(val_loader)
    val_report = classification_report(
        all_val_labels, all_val_preds, output_dict=True, zero_division=0)
    val_precision = val_report['weighted avg']['precision']
    val_recall = val_report['weighted avg']['recall']
    val_f1 = val_report['weighted avg']['f1-score']

    # Save metrics for this epoch
    metrics.append({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "precision": val_precision,
        "recall": val_recall,
        "f1_score": val_f1
    })
    
    print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {train_loss:.4f}, "
          f"Val Loss: {val_loss:.4f}, Precision: {val_precision:.4f}, "
          f"Recall: {val_recall:.4f}, F1-Score: {val_f1:.4f}")
    
    scheduler.step()

# Record the end time of the training process
training_end_time = time.time()
total_training_time_seconds = training_end_time - training_start_time
total_training_time_minutes = total_training_time_seconds / 60

# Save metrics to a CSV file
with open(save_path, 'w', newline='') as csvfile:
    fieldnames = ["epoch", "train_loss", "val_loss", "precision", "recall", "f1_score"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerows(metrics)

# Save total training time in minutes
with open(save_path, 'a', newline='') as csvfile:
    csvfile.write(f"\nTotal Training Time (minutes):,{total_training_time_minutes:.2f}\n")

print(f"Training metrics saved to: {save_path}")
print(f"Total training time for all epochs: {total_training_time_minutes:.2f} minutes")


Epoch [1/30] Train Loss: 1.8066, Val Loss: 1.6728, Precision: 0.5412, Recall: 0.4848, F1-Score: 0.4937
Epoch [2/30] Train Loss: 1.4806, Val Loss: 1.5793, Precision: 0.5711, Recall: 0.5216, F1-Score: 0.5094
Epoch [3/30] Train Loss: 1.3894, Val Loss: 1.3213, Precision: 0.5868, Recall: 0.5479, F1-Score: 0.5415
Epoch [4/30] Train Loss: 1.2193, Val Loss: 1.5107, Precision: 0.5991, Recall: 0.5312, F1-Score: 0.5262
Epoch [5/30] Train Loss: 1.1504, Val Loss: 1.2186, Precision: 0.6174, Recall: 0.5831, F1-Score: 0.5806
Epoch [6/30] Train Loss: 0.9893, Val Loss: 0.9723, Precision: 0.6657, Recall: 0.6597, F1-Score: 0.6516
Epoch [7/30] Train Loss: 0.9121, Val Loss: 0.9389, Precision: 0.7001, Recall: 0.6861, F1-Score: 0.6856
Epoch [8/30] Train Loss: 0.9042, Val Loss: 1.0237, Precision: 0.6860, Recall: 0.6629, F1-Score: 0.6605
Epoch [9/30] Train Loss: 0.8454, Val Loss: 0.8619, Precision: 0.7127, Recall: 0.7029, F1-Score: 0.6947
Epoch [10/30] Train Loss: 0.8236, Val Loss: 0.9872, Precision: 0.6865, Re

### Testing and Evaluation

In [7]:

# Evaluate the model on the test set
model.eval()
all_labels, all_preds = [], []
with torch.no_grad():
    for images, labels, _ in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
report = classification_report(all_labels, all_preds, target_names=train_dataset.classes)
print(report)


              precision    recall  f1-score   support

       delta       0.87      0.92      0.89       330
     estuary       0.84      0.84      0.84       125
       flood       0.81      0.80      0.81       283
    glaciers       0.89      0.94      0.92       240
  hot_spring       0.76      0.83      0.79       201
        lake       0.79      0.82      0.80       153
        pool       0.92      0.92      0.92        79
      puddle       0.74      0.67      0.70       168
      rapids       0.83      0.80      0.82       219
       river       0.78      0.67      0.72        73
         sea       0.82      0.87      0.84       108
        snow       0.85      0.79      0.82       142
       swamp       0.91      0.88      0.89       188
   waterfall       0.74      0.71      0.72        96
     wetland       0.95      0.95      0.95        93

    accuracy                           0.84      2498
   macro avg       0.83      0.83      0.83      2498
weighted avg       0.83   

In [8]:
# Save classification report
with open("classification_report_my_modelwithSGD.txt", "w") as f:
    f.write(report)

# Save model weights
torch.save(model.state_dict(), "my_model_cpu.pth")